In [1]:
# If not already done, install packages. May need to restart kernel after.
!pip install text_lloom pyyaml pandas openpyxl

zsh:1: /Users/ltraum/Documents/GitHub/AmericanPizzaProject/venv/bin/pip: bad interpreter: /Users/ltraum/Documents/GitHub/AmericanPizzaProject/venv311/bin/python3.11: no such file or directory


In [4]:
import sys
import subprocess
import os

# Install all requirements
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])

# Set working directory
os.chdir('/Users/ltraum/Documents/GitHub/AmericanPizzaProject')
print("Working directory set to:", os.getcwd())

# Confirm Python interpreter
print("Python being used:", sys.executable)
print("If this cell runs without error, you're ready to use this notebook 🚀")

Working directory set to: /Users/ltraum/Documents/GitHub/AmericanPizzaProject
Python being used: /Users/ltraum/Documents/GitHub/AmericanPizzaProject/venv/bin/python
If this cell runs without error, you're ready to use this notebook 🚀



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import pandas as pd
import text_lloom.workbench as wb

#setting api key
os.environ["OPENAI_API_KEY"] = ""

In [11]:
#OLD # Replace this path as needed

data_path = "/Users/ltraum/Documents/GitHub/AmericanPizzaProject/pizza_interviews.xlsx"
df = pd.read_excel(data_path)

# Preview -- edit as needed for your sheet format
print(df.columns)
df.head()

Index(['participant_id', 'age', 'city_of_residence', 'state_of_residence',
       'region_of_residence', 'income', 'pizza_consumption',
       'food_restrictions', 'q1_response', 'q2_response', 'q3_response',
       'q4_response', 'q5_response'],
      dtype='object')


,participant_id,age,city_of_residence,state_of_residence,region_of_residence,income,pizza_consumption,food_restrictions,q1_response,q2_response,q3_response,q4_response,q5_response
0,1,32,Boston,Massachusetts,Northeast,$65k,Weekly,No food restrictions,My big pizza moment was trying Regina's in the...,"I want classic Neapolitan-style - thin, chewy ...",Pizza is my social food - Friday nights with m...,Pizza boxes are crucial - there's something sa...,"The biggest barrier is quality, not availabili..."
1,2,45,Atlanta,Georgia,South,$85k,Monthly,No food restrictions,I wouldn't call it a turning point exactly. Gr...,"Depends on my mood, but usually medium-thick c...",Pizza is weekend or special occasion food for ...,"Pizza boxes are part of the ritual, especially...","Atlanta has great availability, but finding pi..."
2,3,67,Miami,Florida,South,$45k,Occasionally eat pizza,No food restrictions,My relationship with pizza has gone through ph...,"At my age, I keep it simple. Classic New York ...",Pizza is mostly about family time now. When gr...,I don't have strong feelings about boxes - the...,The biggest barrier is finding good pizza that...
3,4,23,Chicago,Illinois,Midwest,$28k,Weekly,No food restrictions,I finally tried deep dish at Lou Malnati's sop...,I need to give you two ideals. For deep dish: ...,"As a broke college student, pizza is survival ...",Boxes are essential for sharing and keeping th...,Cost is my biggest barrier. Good deep dish is ...
4,5,41,Birmingham,Alabama,South,$52k,Weekly,Lactose intolerant,"Honestly, I haven't had some big pizza awakeni...","I keep it simple - hand-tossed crust, not too ...",Pizza is family dinner once or twice a week wh...,Pizza boxes are fine - they do the job and kid...,Lactose intolerance is my main barrier now. No...


In [9]:

def filter_demographics(
    df, regions=None, ages=None, income=None, diet=None
):
    df_filtered = df.copy()
    if regions:
        df_filtered = df_filtered[df_filtered["region_of_residence"].isin(regions)]
    if ages:
        df_filtered = df_filtered[df_filtered["age"].isin(ages)]
    if income:
        df_filtered = df_filtered[df_filtered["income"].isin(income)]
    if diet:
        df_filtered = df_filtered[df_filtered["food_restrictions"].isin(diet)]
    return df_filtered.reset_index(drop=True)

# Example: filter to just Northeast region and age 18-40
# filtered = filter_demographics(df, regions=['Northeast'], ages=['18-40'])
# filtered.head()

In [ ]:
#OLD

def get_q4_texts_and_ids(df):
    mask = df['q4_response'].str.strip() != ""
    df_q4 = df.loc[mask]
    texts = df_q4['q4_response'].tolist()
    ids = df_q4['participant_id'].tolist()
    # Optionally also keep extra metadata
    return pd.DataFrame({'participant_id': ids, 'text': texts})

# Prepare for LLoOM:
q4_df = get_q4_texts_and_ids(df)
q4_df.head()

,participant_id,text
0,1,Pizza boxes are crucial - there's something sa...
1,2,"Pizza boxes are part of the ritual, especially..."
2,3,I don't have strong feelings about boxes - the...
3,4,Boxes are essential for sharing and keeping th...
4,5,Pizza boxes are fine - they do the job and kid...


In [12]:

import pandas as pd

# Load your data

df = pd.read_excel(data_path)

# Columns to keep (demographics)
demo_cols = [
    "participant_id",
    "age",
    "city_of_residence",
    "state_of_residence",
    "region_of_residence",
    "income",
    "pizza_consumption",
    "food_restrictions"
]

# All response columns
response_cols = ["q1_response", "q2_response", "q3_response", "q4_response", "q5_response"]

# Add a column that concatenates all Q responses (handles missing by skipping blanks)
df["all_responses"] = df[response_cols].apply(
    lambda row: " ".join([str(r) for r in row if pd.notnull(r) and r.strip() != ""]),
    axis=1
)

# Build q_all_df with all demographics and the concatenated text
q_all_df = df[demo_cols + ["all_responses"]].rename(columns={"all_responses": "text"})
q_4_df = df[demo_cols + ["q4_response"]].rename(columns={"all_responses": "text"})

# Show the columns and a preview
print(q_all_df.columns)
q_all_df.head()

Index(['participant_id', 'age', 'city_of_residence', 'state_of_residence',
       'region_of_residence', 'income', 'pizza_consumption',
       'food_restrictions', 'text'],
      dtype='object')


,participant_id,age,city_of_residence,state_of_residence,region_of_residence,income,pizza_consumption,food_restrictions,text
0,1,32,Boston,Massachusetts,Northeast,$65k,Weekly,No food restrictions,My big pizza moment was trying Regina's in the...
1,2,45,Atlanta,Georgia,South,$85k,Monthly,No food restrictions,I wouldn't call it a turning point exactly. Gr...
2,3,67,Miami,Florida,South,$45k,Occasionally eat pizza,No food restrictions,My relationship with pizza has gone through ph...
3,4,23,Chicago,Illinois,Midwest,$28k,Weekly,No food restrictions,I finally tried deep dish at Lou Malnati's sop...
4,5,41,Birmingham,Alabama,South,$52k,Weekly,Lactose intolerant,"Honestly, I haven't had some big pizza awakeni..."


In [13]:
import asyncio

# Prepare the lloom object
l = wb.lloom(q_all_df, text_col="text", id_col="participant_id")  # change text_col/id_col if yours differ

async def extract_lloom_concepts(l, max_concepts=5, seed="family"):
    # Use gen_auto for one-step themes. seed is optional for steering.
    score_df = await l.gen_auto(max_concepts=max_concepts, seed=seed, debug=False)
    # Export: returns a summary per concept, ready for reporting
    export_df = l.export_df()
    return score_df, export_df

score_df, export_df = await extract_lloom_concepts(l, max_concepts=5)



Estimated cost: $0.12
**Please note that this is only an approximate cost estimate**


Distill-filter
✅ Done    


Distill-summarize
✅ Done    


Cluster
✅ Done    


Synthesize
⠹ Loading 

/Users/ltraum/Documents/GitHub/AmericanPizzaProject/venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/ltraum/Documents/GitHub/AmericanPizzaProject/venv/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ Done    
✅ Done with concept generation!


Active concepts (n=5):
- Family Bonding: Does the text describe pizza as a means to bring family members together or strengthen family bonds?
- Pizza Traditions: Does the text refer to pizza nights or similar traditions as a regular family activity?
- Family Preferences: Does the text mention specific pizza preferences or choices influenced by family members?
- Family Health Concerns: Does the text address health concerns, such as dietary restrictions, that affect family pizza choices?
- Family Celebrations: Does the text refer to food as part of family celebrations or special occasions?


Scoring 5 concepts for 50 documents
Estimated cost: $0.02
**Please note that this is only an approximate cost estimate**
100%|██████████| 5/5 [00:44<00:00,  8.84s/it]
✅ Done with concept scoring!


In [14]:
# Let's see the main outputs tbls
print("Score DataFrame:")
display(score_df.head(10))
print("Exported Concepts/Themes:")
display(export_df.head(10))

Score DataFrame:


,doc_id,text,concept_id,concept_name,concept_prompt,score,rationale,highlight,concept_seed
0,1,Pizza is my social food - Friday nights with m...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text describes pizza as a social food that...,It's comfort food that brings people together ...,family
1,2,Pizza is weekend or special occasion food for ...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text clearly describes pizza as a central ...,Pizza night is sacred family time where we put...,family
2,3,Pizza is mostly about family time now.\nWhen g...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text clearly emphasizes the importance of ...,Pizza nights with family are about conversatio...,family
3,4,my roommate's parents took us to Lou's\nmy roo...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,0.25,The text discusses how pizza is a shared exper...,Pizza brings our friend group together,family
4,5,Pizza is family dinner once or twice a week wh...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text describes pizza as a regular family d...,Pizza is family dinner once or twice a week wh...,family
5,7,"Pizza happens when the grandkids visit, mostly...",34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text emphasizes that pizza is primarily en...,"Pizza happens when the grandkids visit, mostly.",family
6,9,"If my son visits with the grandkids, we might ...",34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text describes ordering pizza when family ...,"If my son visits with the grandkids, we might ...",family
7,11,My wife and I will pick a neighborhood and try...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text describes how the couple chooses to e...,My wife and I will pick a neighborhood and try...,family
8,13,Pizza is family dinner maybe once or twice a m...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text clearly describes pizza night as a sp...,It's comfort food that brings the family toget...,family
9,14,it was an easy dinner solution and they loved ...,34b0b57c-cc36-4fd0-84d3-45931833d3a5,Family Bonding,Does the text describe pizza as a means to bri...,1.00,The text emphasizes the nostalgic and social a...,It's more about the memories and family time t...,family


Exported Concepts/Themes:


,concept,criteria,summary,rep_examples,prevalence,n_matches,highlights
0,Family Bonding,Does the text describe pizza as a means to bri...,"Pizza symbolizes family bonding, creating cher...",[Pizza is my social food - Friday nights with ...,0.52,26,[Good for everything from kid birthday parties...
1,Family Celebrations,Does the text refer to food as part of family ...,Family celebrations often revolve around pizza...,[Pizza is my social food - Friday nights with ...,0.38,19,[It's comfort food that brings the family toge...
2,Family Health Concerns,"Does the text address health concerns, such as...","As we age, dietary preferences shift, impactin...",[Pizza is weekend or special occasion food for...,0.12,6,"[Lactose intolerance is my main barrier now., ..."
3,Family Preferences,Does the text mention specific pizza preferenc...,"Pizza is a family favorite, often chosen for g...",[Pizza is weekend or special occasion food for...,0.54,27,"[In high school, pizza was Friday night family..."
4,Pizza Traditions,Does the text refer to pizza nights or similar...,"Pizza traditions foster family connections, cr...",[My wife and I will pick a neighborhood and tr...,0.42,21,[Pizza is family dinner once or twice a week w...


In [15]:
# Show/report per-theme results as in Streamlit
for i, row in export_df.iterrows():
    print(f"\nTheme {i+1}: {row['concept']}")
    print(f"Criteria: {row['criteria']}")
    print(f"Summary: {row['summary']}")
    print(f"Prevalence: {row['prevalence']*100:.1f}% of participants")
    print("Representative Examples:")
    for ex in row['rep_examples']:
        print("-", ex)
    print("-" * 40)

# Save results to CSV for future dashboard use!
export_df.to_csv("lloom_themes_summary.csv", index=False)
score_df.to_csv("lloom_theme_scores.csv", index=False)


Theme 1: Family Bonding
Criteria: Does the text describe pizza as a means to bring family members together or strengthen family bonds?
Summary: Pizza symbolizes family bonding, creating cherished moments and connections during casual dinners and special gatherings.
Prevalence: 52.0% of participants
Representative Examples:
- Pizza is my social food - Friday nights with my girlfriend watching movies, or hitting spots in the North End with friends before Red Sox games.
I rarely eat it alone or on-the-go.
It's comfort food that brings people together and forces us to slow down during busy weeks.
Weekend afternoons we'll walk to our local place and sit outside with slices and beers.
- My wife and I will pick a neighborhood and try the local pizza place to get a feel for the area's culture.
Pizza is date night or trying new restaurants food.
We probably eat pizza twice a month, always trying somewhere new rather than having regular spots.
The only barrier is decision paralysis - Houston ha

In [16]:
#l.vis()
l.vis(slice_col="region_of_residence")



MatrixWidget(data='[{"id":"All","value":26,"example":"All","_my_score":0,"concept":"Family Bonding","n":26},{"…

In [17]:
# Example: Find themes relevant to a given participant
pid = export_df['rep_examples'][0][0]  # Just a sample; use as needed

participant_themes = score_df[score_df['participant_id']==pid]
display(participant_themes)

KeyError: 'participant_id'